# RoBERTa for IMDB Sentiment Classification: 

## Motivation: 

After utilizing BERT and the continuous evolvement within the field as discussed in class, I wanted to try out RoBERTa as well for the task at hand. RoBERTa stands for Robustly Optimized BERT Pretraining Approach. RoBERTa is trained on a much larger corpus than BERT, approxiamtely 170,000 books and various forums across the internet. It modifies key hyperparameters of BERT and has a larger batch size and learning rate. Given these features of RoBERTa, the motivation would be to better capture the semantic context of sentences and lead to better predictions. 

The approach below is very similar to the BERT notebook. I use tensorflow to utilise RoBERTa as well. 

Link: [https://huggingface.co/docs/transformers/model_doc/roberta]

In [1]:
# install transformers
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 36.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 27.8 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 52.6 MB/s eta 0:00:00


In [1]:
# load: 
import os 
import json
import numpy as np
import pandas as pd
import re
from tqdm import tqdm
import time
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
# load the pre-trained models: 
from transformers import RobertaTokenizer, TFRobertaModel
with tf.device('/device:GPU:0'):
    tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
    model = TFRobertaModel.from_pretrained("roberta-base")

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


In [3]:
# load csvs: 
train_path = '/home/ec2-user/SageMaker/Kartik_Amazon_Thesis/general_training_data'
read_reviews = pd.read_csv(os.path.join(train_path,'read_reviews_general.csv'))
read_reviews.columns

Index(['review_id', 'full_review', 'wish_I_read_reviews ',
       'wish_I_read_reviews _ohe'],
      dtype='object')

In [4]:
read_reviews['wish_I_read_reviews _ohe'].value_counts()

0    9344
1     516
Name: wish_I_read_reviews _ohe, dtype: int64

In [5]:
read_reviews[read_reviews['wish_I_read_reviews _ohe']==1].iloc[80].full_review

'Great size and wonderful organizer however the back split in 6 months.  Wish i had read more reviews'

In [6]:
read_reviews= read_reviews[['wish_I_read_reviews _ohe', 'full_review']]

In [7]:
# train test split - split trainning into train and validation, take 10% as validation
train, test_df = train_test_split(read_reviews, test_size= 0.3, random_state=42, stratify=read_reviews['wish_I_read_reviews _ohe'])

In [14]:
# train test split - split trainning into train and validation, take 10% as validation
train_df_1, val_df = train_test_split(train, test_size= 0.2, random_state=42, stratify=train['wish_I_read_reviews _ohe'])

In [16]:
val_df['wish_I_read_reviews _ohe'].value_counts()

0    1309
1      72
Name: wish_I_read_reviews _ohe, dtype: int64

In [17]:
val_df.iloc[1].full_review

'Missing parts and a scratched front and chipped corner. Many other reviews with same issue.  Expect to be disappointed.'

In [18]:

class create_tensor_dataset(): 
    
    '''Purpose: Create a tensorflow dataset for model training
    df_to_convert: The dataframe of interest, that contains the 
    text columns
    text: The column within the dataframe containing the text 
    batch_size: int, batch size
    tokenizer: The tokenizer to be utilised
    labels: The class/label column within the dataframe
    '''
    
    def __init__(self, df_to_convert, text,target_col, batch_size, labels=None ): 
        self.df_to_convert = df_to_convert
        self.text = text # column containing the text (here chunked_data)
        self.target_col = target_col # y column of interest here label
        self.batch_size = batch_size
        self.tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
        self.labels = labels # list of all the labels 
        
    def generate_training_data(self, ids, masks):
        for i, text in tqdm(enumerate(self.df_to_convert[self.text])):
            with tf.device('/device:GPU:0'):
                
                tokenized_text =  self.tokenizer.encode_plus(
                    text,# The sequence or batch of sequences to be encoded.
                    max_length=256, #Controls the maximum length to use by one of the truncation/padding parameters.
                    truncation=True, # Truncate to a maximum length specified with the argument max_length
                    padding='max_length', #pad to a maximum length specified with the argument max_length 
                    add_special_tokens=True,# encode the sequences with the special tokens relative to their model.
                    return_tensors='tf'   #return TensorFlow tf.constant objects.
                )
                ids[i, :] = tokenized_text.input_ids ##List of token ids to be fed to a model
                masks[i, :] = tokenized_text.attention_mask #List of indices specifying which tokens should be attended to by the model
        return ids, masks
    
    def ohe_label(self): 
         # convert label values to a list 
        return self.df_to_convert[self.target_col].values.tolist()

    def DatasetMapFunction(self, input_ids, attn_masks, labels):
        # create and return key-value pair of input ids, attention mask and the corresnponding labels
        return {
            'input_ids': input_ids,
            'attention_mask': attn_masks
        }, labels
    
    def tensor_data(self): 
        # input_ids array
        X_input_ids = np.zeros((len(self.df_to_convert), 256))
        # attention mask array
        X_attn_masks = np.zeros((len(self.df_to_convert), 256))
        # retrieve the input ids and attention masks
        X_input_ids, X_attn_masks = self.generate_training_data(X_input_ids, X_attn_masks)
        # creation of tensorflow dataset: 
        dataset = tf.data.Dataset.from_tensor_slices((X_input_ids, X_attn_masks, self.ohe_label()))
        dataset = dataset.map(self.DatasetMapFunction)
        dataset = dataset.shuffle(10000).batch(self.batch_size, drop_remainder=True)
        return dataset

In [19]:
# creation of tensorflow dataset for training
import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)
train_tensor =create_tensor_dataset(train_df_1, 'full_review', 'wish_I_read_reviews _ohe' ,8 )
train_dataset =train_tensor.tensor_data() 

5521it [00:07, 786.91it/s]


In [20]:
# validation tensor dataset: 
with tf.device('/device:GPU:0'):
    val_tensor =create_tensor_dataset(val_df, 'full_review', 'wish_I_read_reviews _ohe', 8)
    val_dataset =val_tensor.tensor_data() 

1381it [00:01, 748.82it/s]


In [21]:

# defining 2 input layers for input_ids and attn_masks
with tf.device('/device:GPU:0'):
    input_ids = tf.keras.layers.Input(shape=(256,), name='input_ids', dtype='int32')
    attn_masks = tf.keras.layers.Input(shape=(256,), name='attention_mask', dtype='int32')
    bert_embds = model.roberta(input_ids, attention_mask=attn_masks)[1] # pooled output
    # intermediate_layer_1 = tf.keras.layers.Dropout(0.5, name="dropout")(bert_embds)# 0 -> activation layer (3D), 1 -> pooled output layer (2D)
    intermediate_layer = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer')(bert_embds)
    # intermediate_layer_2 = tf.keras.layers.Dropout(0.4, name="dropout2")(intermediate_layer)
    output_layer = tf.keras.layers.Dense(1, activation='sigmoid', name='output_layer')(intermediate_layer) # sigmoid -> calcs probs of binary classes
    model = tf.keras.Model(inputs=[input_ids, attn_masks], outputs=output_layer)
    model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 roberta (TFRobertaMainLayer)   TFBaseModelOutputWi  124645632   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                           

In [22]:
# utilization of adam optimization 
optim = tf.keras.optimizers.Adam(learning_rate=1e-5)
# utilised binary cross entropy loss 
loss_func = tf.keras.losses.BinaryCrossentropy()
model.compile(optimizer=optim, loss=loss_func, metrics=['accuracy'])

# early stopping - administer val_loss: 
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping()
custom_early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience= 3,
    mode='min'
)

with tf.device('/device:GPU:0'):

    hist = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=100, 
        callbacks =[custom_early_stopping]
    )

Epoch 1/100
690/690 [==============================] - 357s 472ms/step - loss: 0.1208 - accuracy: 0.9649 - val_loss: 0.0526 - val_accuracy: 0.9818
Epoch 2/100
690/690 [==============================] - 319s 463ms/step - loss: 0.0480 - accuracy: 0.9853 - val_loss: 0.0400 - val_accuracy: 0.9855
Epoch 3/100
690/690 [==============================] - 317s 460ms/step - loss: 0.0333 - accuracy: 0.9902 - val_loss: 0.0610 - val_accuracy: 0.9833
Epoch 4/100
690/690 [==============================] - 317s 460ms/step - loss: 0.0236 - accuracy: 0.9935 - val_loss: 0.0521 - val_accuracy: 0.9840
Epoch 5/100
690/690 [==============================] - 317s 459ms/step - loss: 0.0173 - accuracy: 0.9946 - val_loss: 0.0571 - val_accuracy: 0.9833


In [12]:
def prepare_data(input_text, tokenizer):
    with tf.device('/device:GPU:0'):
        
        token = tokenizer.encode_plus(
            input_text,
            max_length=256, 
            truncation=True, 
            padding='max_length', 
            add_special_tokens=True,
            return_tensors='tf'
        )
        return {
            'input_ids': tf.cast(token.input_ids, tf.float64),
            'attention_mask': tf.cast(token.attention_mask, tf.float64)
        }

def make_predictions(model, input_text,threshold, label_list=None ): 
    with tf.device('/device:GPU:0'):
        processed_data = prepare_data(input_text, tokenizer)
        probs = model.predict(processed_data)
#         print(probs)
        if probs[0]> threshold: 
            return label_list[0]
        else: 
            return label_list[1]
#     return(probs[0])

In [24]:
# tt = test_df.head(20)
label_list= [1, 0]
test_df['result']= test_df.full_review.apply(lambda x: make_predictions(model, x, 0.5, label_list))

1/1 [==============================] - 0s 61ms/step


1/1 [==============================] - 0s 60ms/step


1/1 [==============================] - 0s 60ms/step


1/1 [==============================] - 0s 63ms/step


1/1 [==============================] - 0s 59ms/step


1/1 [==============================] - 0s 61ms/step


1/1 [==============================] - 0s 60ms/step


1/1 [==============================] - 0s 59ms/step


1/1 [==============================] - 0s 59ms/step


1/1 [==============================] - 0s 61ms/step


In [25]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
print (classification_report(test_df['wish_I_read_reviews _ohe'], test_df['result']))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      2803
           1       0.78      0.88      0.83       155

    accuracy                           0.98      2958
   macro avg       0.89      0.93      0.91      2958
weighted avg       0.98      0.98      0.98      2958



In [19]:
roberta_imdb.save('read_reviews_v1')

INFO:tensorflow:Assets written to: read_reviews_v1/assets


INFO:tensorflow:Assets written to: read_reviews_v1/assets


In [22]:
test_df[test_df['result']==1].iloc[4].full_review

"This may work on carpet, but it will not work on hard surfaces.  Extremely dangerous!  I tried on stained concrete floor (with anti-skid) and it will literally slide out from under you.  I'm convinced it moves even while not in use.....  I should have paid more attention to the reviews, that said said that slippage, was an issue!  The other problem with it, it will not lay flat.  The edges curl up.  If you flip every other day, it will lay flat.  I ordered another brand at the same time (for another desk).  The other one does move some, but not at all like this one.  The other brand also lays flat."

## Conclusion: 

The RoBERTa model was able to achieve a accuracy of ~95% on the testing dataset. It can also be seen that there is a lot more parameters involved in comparison to BERT embeddings. The intresting feature of RoBERTa is that it can take a max length of upto 1024, however, given the sentence length distribution for the task at hand, max length remained at 512. Like BERT, it is able to predict significantly in comparison to other models but remains extremely computationally expensive for the task at hand. I utilised the pooled output from the roberta model for the task at hand too. 

Accuracy achieved - 95%. 

In [14]:
model = tf.keras.models.load_model('/home/ec2-user/SageMaker/models_office_v1/read_reviews_v1')
# os.getcwd()

In [15]:
# tt = test_df.head(20)
label_list= [1, 0]
test_df['result']= test_df.full_review.apply(lambda x: make_predictions(model, x, 0.5, label_list))

1/1 [==============================] - 0s 41ms/step


1/1 [==============================] - 0s 42ms/step


1/1 [==============================] - 0s 41ms/step


1/1 [==============================] - 0s 42ms/step


1/1 [==============================] - 0s 42ms/step


In [16]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
print (classification_report(test_df['wish I read reviews _ohe'], test_df['result']))

              precision    recall  f1-score   support

           0       0.98      0.99      0.98      1324
           1       0.83      0.81      0.82       115

    accuracy                           0.97      1439
   macro avg       0.91      0.90      0.90      1439
weighted avg       0.97      0.97      0.97      1439



In [21]:
test_df[test_df['wish I read reviews _ohe']==1].iloc[3].full_review

'DON\'T BUY THIS CHEAP PIECE OF .... Just read the reviews. The "stainless steel" cable makes it sound heavy, duty but it\'s not. Like at least one other reviewer, I found that the cheap stitching on the belt came apart within a couple weeks with minimal use.'

In [33]:
 a= test_df[test_df['result']==0].iloc[3].full_review

In [15]:
test_df

,wish_I_read_reviews _ohe,full_review
5051,0,The premise and promise of adventure with a fa...
8727,0,Removed a few staples before the thick point b...
2295,0,I'm a big Lennon fan including his work with T...
8259,0,There were a few Smash accessories I wanted an...
7610,0,I bought 3 stamps and today was the first time...
...,...,...
9445,0,I am giving this one star because of one main ...
4311,0,Lots of star power... but doesn't tie together...
8292,0,Toners arrived poorly wrapped inside a bag(one...
5817,0,Luckily I did not waste any money on this book...


In [9]:
os.getcwd()

'/home/ec2-user/SageMaker/Kartik_Amazon_Thesis/read_reviews'

In [14]:
## testing office model on office plus general population: 

In [11]:
model= tf.keras.models.load_model('/home/ec2-user/SageMaker/best_models/read_reviews_v1')

In [ ]:
# tt = test_df.head(20)
label_list= [1, 0]
test_df['result']= test_df.full_review.apply(lambda x: make_predictions(model, x, 0.5, label_list))

1/1 [==============================] - 0s 60ms/step


1/1 [==============================] - 0s 58ms/step


1/1 [==============================] - 0s 59ms/step


1/1 [==============================] - 0s 58ms/step


1/1 [==============================] - 0s 59ms/step


...

1/1 [==============================] - 0s 59ms/step


1/1 [==============================] - 0s 59ms/step


1/1 [==============================] - 0s 59ms/step


1/1 [==============================] - 0s 61ms/step


1/1 [==============================] - 0s 59ms/step


In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
print (classification_report(test_df['wish_I_read_reviews _ohe'], test_df['result']))